In [36]:
import pandas as pd
import json
import os
import cv2
import pandas as pd
import numpy as np

In [74]:
# read annotation files
data = json.load(open('annotations.json'))
data = data['imgs']

In [75]:
# store them
annotations = {}
for key, value in data.items():
    if len(value['objects']) != 0:
        path = value['path']
        objects = []
        for e in value['objects']:
            objects.append(e['bbox'])
            
        annotations[key] = objects

In [31]:
# check img sizes
im_names = []
sizes = []
for e in os.listdir('./other'):
    im_names.append(e)
    im = cv2.imread(f'./other/{e}')
    h, w, c = im.shape
    # in this case: h,w = 2048
    sizes.append([h,w])

# are they all the same?
pd.DataFrame(sizes).value_counts()

KeyboardInterrupt: 

In [76]:
# {'imgage name' : [[obj1], [obj2], [objn]] }
objects = {}
for key, obj in annotations.items():
    obj_list = []
    for e in obj:
        # [xmin, ymin, ymax, xmax]
        # obj_list.append([e['xmin'], e['ymin'], e['ymax'], e['xmax']])
        # [xcenter, ycenter, objwidth, objheight]
        obj_list.append([0, (e['xmin'] + e['xmax']) / 2*2048, (e['ymin'] + e['ymax']) / 2*2048, (e['ymax'] - e['ymin']) / 2048, (e['xmax'] - e['xmin']) / 2048])
    objects[key] = obj_list

In [77]:
# save them as txt
fmt = '%.10g'

for name, obj in objects.items():
    np.savetxt(f'./other/{name}.txt', obj, newline='\n',fmt=fmt)

In [79]:
# because of our corrupted file there are some mismatches between .jpg and .txt files, we want to pair them and delete th rest
for e in os.listdir('./other'):
    if (e.endswith('.txt')):
        if e.replace(e[-4:],'.jpg') not in os.listdir('./other'):
            os.remove(f'./other/{e}')

In [80]:
for e in os.listdir('./other'):
    if (e.endswith('.jpg')):
        if e.replace(e[-4:],'.txt') not in os.listdir('./other'):
            os.remove(f'./other/{e}')